In [9]:
import torch
import sys
import os
from pathlib import Path
import csv
import yaml

# append the current path
ROOT = Path.cwd().resolve().parents[0]
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from utils import *
from utils import get_all_preds, plot_confmat, plot_gradcam, plot_results
from dataset.data import get_transforms, load_dataset
from train import get_num_correct


In [10]:
# checkpoints files path
def get_ckpt(model, get_state_dict = False):
    ckpt = f"../weights/Runs/weights/lr3e-5_{model}_cuda.pth"
    if get_state_dict:
        model_info = torch.load(ckpt, map_location= torch.device("cpu"))
        epoch = model_info["epoch"]
        model_sd = model_info["model_state_dict"]
        return ckpt, model_sd
    return ckpt


In [11]:
# test path..
path = get_ckpt("resnet18")
if os.path.exists(path):
    print("Yes, the path exits")
else:
    print("Oops! the path doesnt exists")


Yes, the path exits


In [12]:
config_file = "../configs/configs.yaml"
with open(config_file, "r") as f:
    cfg = yaml.safe_load(f)
# print(cfg)

In [13]:
# set the device
device = "cuda" if torch.cuda.is_available() else "cpu"

In [14]:
# data loaders
train_loader = load_dataset(config_file= cfg, batch_size= 32, 
                            kind= "train", drop_last = False)
test_loader = load_dataset(config_file=cfg, batch_size=32, 
                           kind = "test", drop_last = False)

### pick the model
- loading the ckpt 
- evaluating resnet18 performance
- plotting results and confusion matrix

In [24]:
updated_model = get_model("densenet121", pretrained=False,
                      num_classes=4, weights= get_ckpt("densenet121", get_state_dict=True)[1])


In [25]:
train_predictions, train_labels = get_all_preds(updated_model, train_loader)
test_predictions, test_labels = get_all_preds(updated_model, test_loader)

In [26]:
# check dimensions of each prdiction
print("Training prediction: {}".format(train_predictions.shape))
print("Test predictions: {}".format(test_predictions.shape))

Training prediction: torch.Size([20685, 4])
Test predictions: torch.Size([240, 4])


In [27]:
train_corrects = get_num_correct(train_predictions, train_labels )
test_corrects = get_num_correct(test_predictions, test_labels)

print(f'Train Correct: {train_corrects:5}\tTrain Accuracy: {(100*train_corrects/len(train_loader.dataset)):5.2f}%')
print(f'Test Correct: {test_corrects:6}\tTest Accuracy: {(100*test_corrects/len(test_loader.dataset)):6.2f}%')

Train Correct: 20601	Train Accuracy: 99.59%
Test Correct:    231	Test Accuracy:  96.25%


In [28]:
train_confmat =get_confmat(train_labels, train_predictions)
test_confmat = get_confmat(test_labels, test_predictions)
# plot_confmat(train_confmat, test_confmat, classes=train_loader.dataset.classes, filename=f'{type(updated_model).__name__.lower()}')


In [29]:
results = get_results(test_confmat, test_loader.dataset.classes)
results

{'covid_19': [0.9871794871794872, 1.0, 0.95, 0.9743589743589743],
 'lung_opacity': [0.9665271966527197,
  0.9333333333333333,
  0.9333333333333333,
  0.9333333333333333],
 'normal': [0.9746835443037974,
  0.9354838709677419,
  0.9666666666666667,
  0.9508196721311476],
 'pneumonia': [0.9956896551724138,
  0.9836065573770492,
  1.0,
  0.9917355371900827]}

In [30]:
import pandas as pd
df = pd.DataFrame(results)
model = 'densenet121'
df.index = ['accuracy', 'recall', 'precision', 'f1score']
df.to_csv(f'../runs/{model}.csv')

In [31]:
df

,covid_19,lung_opacity,normal,pneumonia
accuracy,0.987179,0.966527,0.974684,0.995690
recall,1.000000,0.933333,0.935484,0.983607
precision,0.950000,0.933333,0.966667,1.000000
f1score,0.974359,0.933333,0.950820,0.991736


TypeError: bark() takes 0 positional arguments but 1 was given